[View in Colaboratory](https://colab.research.google.com/github/wonwooddo/Pytorch_tutorial/blob/master/02.intermediate_CNN.ipynb)

In [8]:
!pip install torch
!pip install torchvision

In [0]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [0]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [0]:

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

In [0]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [0]:

model = ConvNet(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [15]:

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.1494
Epoch [1/5], Step [200/600], Loss: 0.1661
Epoch [1/5], Step [300/600], Loss: 0.1476
Epoch [1/5], Step [400/600], Loss: 0.0349
Epoch [1/5], Step [500/600], Loss: 0.0557
Epoch [1/5], Step [600/600], Loss: 0.0661
Epoch [2/5], Step [100/600], Loss: 0.0501
Epoch [2/5], Step [200/600], Loss: 0.0821
Epoch [2/5], Step [300/600], Loss: 0.1572
Epoch [2/5], Step [400/600], Loss: 0.0513
Epoch [2/5], Step [500/600], Loss: 0.0225
Epoch [2/5], Step [600/600], Loss: 0.0284
Epoch [3/5], Step [100/600], Loss: 0.0097
Epoch [3/5], Step [200/600], Loss: 0.0355
Epoch [3/5], Step [300/600], Loss: 0.0223
Epoch [3/5], Step [400/600], Loss: 0.0232
Epoch [3/5], Step [500/600], Loss: 0.0281
Epoch [3/5], Step [600/600], Loss: 0.0333
Epoch [4/5], Step [100/600], Loss: 0.0631
Epoch [4/5], Step [200/600], Loss: 0.0213
Epoch [4/5], Step [300/600], Loss: 0.0362
Epoch [4/5], Step [400/600], Loss: 0.1039
Epoch [4/5], Step [500/600], Loss: 0.0260
Epoch [4/5], Step [600/600], Loss:

In [16]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))


Test Accuracy of the model on the 10000 test images: 99 %


In [0]:

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')